In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fraud-data/sample_submission.csv
/kaggle/input/fraud-data/test_identity.csv
/kaggle/input/fraud-data/train_identity.csv
/kaggle/input/fraud-data/test_transaction.csv
/kaggle/input/fraud-data/train_transaction.csv


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GroupShuffleSplit
from sklearn.utils import resample
import shap

In [7]:
!pip install dagshub mlflow -q

In [8]:
import dagshub
dagshub.init(repo_owner='gkuch22', repo_name='ml-hw2-gkuch22', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=b49f1c8b-2a5e-41e4-bbd8-cdfa1719c455&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=2089c6c454c70d05a8fc02be2bbb30d9c7c2d4da932b4ffa04aaff79b80fc613




Accessing as gkuch22

Initialized MLflow to track repo "gkuch22/ml-hw2-gkuch22"

Repository gkuch22/ml-hw2-gkuch22 initialized!

In [9]:
submission = pd.read_csv('/kaggle/input/fraud-data/sample_submission.csv')
submission

,TransactionID,isFraud
0,3663549,0.5
1,3663550,0.5
2,3663551,0.5
3,3663552,0.5
4,3663553,0.5
...,...,...
506686,4170235,0.5
506687,4170236,0.5
506688,4170237,0.5
506689,4170238,0.5


In [10]:
df_test_transaction = pd.read_csv('/kaggle/input/fraud-data/test_transaction.csv')
df_test_identity = pd.read_csv('/kaggle/input/fraud-data/test_identity.csv')

df_test = df_test_transaction.merge(df_test_identity, how='left', on='TransactionID')
df_test['key_id'] = (
    df_test['card1'].astype(str) + '_' +
    df_test['addr1'].astype(str) + '_' +
    df_test['P_emaildomain'].fillna('unknown').astype(str) + '_' +
    df_test['DeviceType'].astype(str)
)
df_test_transaction.shape, df_test_identity.shape, df_test.shape

df_test.columns = [col.replace('-', '_') for col in df_test.columns]

In [13]:
import mlflow

import os
os.environ["MLFLOW_TRACKING_USERNAME"] = "gkuch22"
os.environ["MLFLOW_TRACKING_PASSWORD"] = ""

mlflow.set_tracking_uri("https://dagshub.com/gkuch22/ml-hw2-gkuch22.mlflow")

logged_model = "models:/XGBoost_BM/1"

loaded_model = mlflow.sklearn.load_model(logged_model)

In [14]:
y_test_proba = loaded_model.predict_proba(df_test)[:, 1]

In [15]:
y_test_proba

array([0.03385817, 0.05782121, 0.15815084, ..., 0.09095923, 0.11985089,
       0.06877183], dtype=float32)

In [16]:
submission = pd.DataFrame({
    'TransactionID': df_test['TransactionID'],
    'isFraud': y_test_proba
})

In [17]:
submission

,TransactionID,isFraud
0,3663549,0.033858
1,3663550,0.057821
2,3663551,0.158151
3,3663552,0.030115
4,3663553,0.061916
...,...,...
506686,4170235,0.103303
506687,4170236,0.121072
506688,4170237,0.090959
506689,4170238,0.119851


In [18]:
submission.to_csv('xgboost_submission.csv', index=False)